<a href="https://colab.research.google.com/github/leehimcm/AI-SS-ASSISTANT-AI/blob/main/endpoint_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* main.py 테스트


### 라이브러리 설치

In [1]:
!pip install pulp gymnasium==0.28.1 tianshou==1.2.0 numpy==1.26.4 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.1/273.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 847.8/847.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 9.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0

### import

In [1]:
import random
from datetime import date, datetime, timedelta, time
from typing import List, Dict, Optional, Tuple
from collections import defaultdict, deque

import numpy as np
import pulp

import torch
import torch.nn as nn

import gymnasium as gym
from gymnasium import spaces

import tianshou as ts
from tianshou.env import DummyVectorEnv
from tianshou.utils.net.common import Net
from tianshou.utils.net.discrete import Actor, Critic
from tianshou.policy import PPOPolicy
from tianshou.data import Collector
from torch.distributions import Categorical, Distribution

from tianshou.trainer import OnpolicyTrainer

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### 내부 데이터
* user_info: 온보딩 데이터
*   cleaning_tasks: 청소 관련 정보 포함됨, 수정되지 않는 데이터
*   user_cleaning_status: 각 청소의 마지막 수행 일자 (앱에서 받아 온 청소 완료 기록 데이터 기반으로 수정됨)
* user_execution_data: 과거 청소 기록 데이터




In [47]:
raw_user_info = {
  'user_nickname': '김청소',
  'user_address': '서울특별시 서대문구 대신동',
  'survey_responses': {
    'cleaning_frequency': 'DAILY',
    'cleaning_distribution': 'SPLIT_AND_MAINTAIN',
    'preferred_time_range': 'MORNING',
    'preferred_days': ['TUE', 'THU', 'SAT'],
    'item_quantity': 'TOO_MUCH',
    'sleep_at': 22,
    'wake_at': 8
  }
}

In [48]:
def transform_user_info_numeric(raw_user_info):
    freq_map = {
        "DAILY": 7/7,        # 매일
        "FREQUENT": 4/7,     # 주 3~5회 → 평균 4회
        "OCCASIONAL": 1.5/7, # 주 1~2회 → 평균 1.5회
        "BIWEEKLY": 0.5/7,   # 2주 1회
        "MONTHLY": 0.25/7    # 한 달 1회
    }
    dist_map = {
        "SPLIT_AND_MAINTAIN": [1.0, 0.0],
        "ONE_TIME_DEEP": [0.0, 1.0]
    }
    time_map = {
        "DAWN": [0, 6],       # 00:00-06:00
        "MORNING": [6, 12],   # 06:00-12:00
        "AFTERNOON": [12, 18],# 12:00-18:00
        "EVENING": [18, 24]   # 18:00-24:00
    }
    day_map = {
        "MON": 0,  # 월요일
        "TUE": 1,  # 화요일
        "WED": 2,  # 수요일
        "THU": 3,  # 목요일
        "FRI": 4,  # 금요일
        "SAT": 5,  # 토요일
        "SUN": 6   # 일요일
    }
    quantity_map = {
        "LITTLE": 0.3,     # 물건이 적어서 정리가 쉬워요
        "TOO_MUCH": 1.0    # 물건이 많아서 정리가 어려워요
    }
    survey = raw_user_info["survey_responses"]
    transformed = {
        "user_nickname": raw_user_info["user_nickname"],
        "user_address": raw_user_info["user_address"],
        "survey_responses": {
            "cleaning_frequency": freq_map.get(survey["cleaning_frequency"], None),
            "cleaning_distribution": dist_map.get(survey["cleaning_distribution"], None),
            "preferred_time_range": time_map.get(survey["preferred_time_range"], None),
            "preferred_days": [day_map[d] for d in survey["preferred_days"]],
            "item_quantity": quantity_map.get(survey["item_quantity"], None),
            "sleep_at": survey.get("sleep_at"),
            "wake_at": survey.get("wake_at")
        }
    }
    return transformed

user_info = transform_user_info_numeric(raw_user_info)


In [49]:
user_info

{'user_nickname': '김청소',
 'user_address': '서울특별시 서대문구 대신동',
 'survey_responses': {'cleaning_frequency': 1.0,
  'cleaning_distribution': [1.0, 0.0],
  'preferred_time_range': [6, 12],
  'preferred_days': [1, 3, 5],
  'item_quantity': 1.0,
  'sleep_at': 22,
  'wake_at': 8}}

In [57]:
# cleaning_tasks: 청소 관련 정보 포함됨, 수정되지 않는 데이터
cleaning_tasks = [
    {"주방 - 수건 교체": {"duration": 5 / 60, "interval": 5, "description": "깨끗한 수건으로 교체 후 사용한 수건은 세탁 바구니에 넣기"}},
    {"주방 - 행주 삶기": {"duration": 10 / 60, "interval": 7, "description": "행주를 삶거나 세탁기에 고온세탁하여 위생 관리"}},
    {"주방 - 배기후드 기름떼 제거": {"duration": 20 / 60, "interval": 30, "description": "기름 제거제를 뿌리고 문질러 닦은 후 물로 헹구기"}},
    {"주방 - 냉장고 유통기한 지난 음식 1개 이상 처분": {"duration": 10 / 60, "interval": 7, "description": "유통기한 지난 식품 확인 후 폐기"}},

    {"세탁실 - 세탁기 세제통 씻기": {"duration": 10 / 60, "interval": 30, "description": "세제 찌꺼기를 제거하고 물로 헹군 후 건조"}},

    {"화장실 - 바닥 청소": {"duration": 10 / 60, "interval": 14, "description": "바닥에 락스를 뿌리고 청소솔과 세제로 구석구석 닦기"}},
    {"화장실 - 변기 청소": {"duration": 10 / 60, "interval": 7, "description": "변기 전용 세제로 내부와 테두리 문지르기"}},
    {"화장실 - 세면대 청소": {"duration": 7 / 60, "interval": 5, "description": "물때와 치약 자국 등을 세정제로 제거"}},
    {"화장실 - 하수구 청소": {"duration": 10 / 60, "interval": 7, "description": "막힘 방지용 세정제 또는 끓는 물로 청소"}},
    {"화장실 - 거울 닦기": {"duration": 5 / 60, "interval": 7, "description": "유리세정제로 물자국 닦기"}},
    {"화장실 - 수납장 내부 정리 및 청소": {"duration": 15 / 60, "interval": 90, "description": "모든 물건 꺼내고 내부를 닦은 후 재정리"}},

    {"거실 - 가구 표면 먼지 닦기 (물걸레)": {"duration": 5 / 60, "interval": 14, "description": "물에 적신 걸레로 가구 표면 닦기"}},
    {"거실 - 가구 표면 먼지 털기 (먼지털이)": {"duration": 5 / 60, "interval": 7, "description": "먼지털이개로 가볍게 먼지를 털어냄"}},
    {"거실 - 바닥 물걸레질": {"duration": 15 / 60, "interval": 3, "description": "청소기로 먼지 제거 후 물걸레질"}},
    {"거실 - 청소기로 바닥 먼지 제거": {"duration": 10 / 60, "interval": 2, "description": "진공청소기로 바닥과 카펫 먼지 제거"}},
    {"거실 - 테이블 위 정리": {"duration": 5 / 60, "interval": 2, "description": "불필요한 물건 정리 및 정돈"}},
    {"거실 - 안 쓰는 물건 1개 이상 처분": {"duration": 7 / 60, "interval": 10, "description": "수납공간 내 불필요한 물건 정리 및 기부/폐기"}},

    {"창틀 먼지 닦기": {"duration": 10 / 60, "interval": 14, "description": "젖은 걸레나 진공청소기를 이용해 틈새 먼지 제거"}},
    {"전자기기 소독": {"duration": 5 / 60, "interval": 5, "description": "알콜스왑이나 알콜티슈로 스마트폰, 컴퓨터 등 표면 닦기"}},
    {"쓰레기 배출": {"duration": 10 / 60, "interval": 3, "description": "생활 쓰레기 및 분리수거된 품목을 지정 장소에 배출"}},
    {"음식물 쓰레기 배출": {"duration": 5 / 60, "interval": 3, "description": "음식물 쓰레기를 지정 장소에 배출"}},
]

# user_cleaning_status: 사용자 청소 기록 데이터 (앱에서 받아 온 청소 완료 기록 데이터 기반으로 수정됨)
# 자동 생성 규칙: last_done_days_ago: cleaning_tasks의 interval + 3 값으로, 첫 사용시에만 자동 생성
def init_user_cleaning_status(cleaning_tasks):
    status = {}
    for task in cleaning_tasks:
        name = list(task.keys())[0]
        meta = task[name]
        interval = meta.get("interval", 0)
        status[name] = {"last_done_days_ago": interval + 3}
    return status

user_cleaning_status = init_user_cleaning_status(cleaning_tasks)



In [58]:
pprint(user_cleaning_status)

{'거실 - 가구 표면 먼지 닦기 (물걸레)': {'last_done_days_ago': 17},
 '거실 - 가구 표면 먼지 털기 (먼지털이)': {'last_done_days_ago': 10},
 '거실 - 바닥 물걸레질': {'last_done_days_ago': 6},
 '거실 - 안 쓰는 물건 1개 이상 처분': {'last_done_days_ago': 13},
 '거실 - 청소기로 바닥 먼지 제거': {'last_done_days_ago': 5},
 '거실 - 테이블 위 정리': {'last_done_days_ago': 5},
 '세탁실 - 세탁기 세제통 씻기': {'last_done_days_ago': 33},
 '쓰레기 배출': {'last_done_days_ago': 6},
 '음식물 쓰레기 배출': {'last_done_days_ago': 6},
 '전자기기 소독': {'last_done_days_ago': 8},
 '주방 - 냉장고 유통기한 지난 음식 1개 이상 처분': {'last_done_days_ago': 10},
 '주방 - 배기후드 기름떼 제거': {'last_done_days_ago': 33},
 '주방 - 수건 교체': {'last_done_days_ago': 8},
 '주방 - 행주 삶기': {'last_done_days_ago': 10},
 '창틀 먼지 닦기': {'last_done_days_ago': 17},
 '화장실 - 거울 닦기': {'last_done_days_ago': 10},
 '화장실 - 바닥 청소': {'last_done_days_ago': 17},
 '화장실 - 변기 청소': {'last_done_days_ago': 10},
 '화장실 - 세면대 청소': {'last_done_days_ago': 8},
 '화장실 - 수납장 내부 정리 및 청소': {'last_done_days_ago': 93},
 '화장실 - 하수구 청소': {'last_done_days_ago': 10}}


In [51]:
user_execution_data = {}
'''
{
   1: [...],
   2: [...],
   ...
}
'''

'\n{\n   1: [...],\n   2: [...],\n   ...\n}\n'

### data_processing.py

*   compute_user_cleaning_status
*   compute_user_behavior
*   preprocessing_schedules
*   prepare_user_state



In [7]:
def compute_user_cleaning_status(user_cleaning_status, prev_week_execution):
    user_cleaning_status = user_cleaning_status or {}
    not_done = set(list(user_cleaning_status.keys()))
    for entry in prev_week_execution:
        if not entry['performed']:
            continue
        task = entry['task']
        prev_days_ago = user_cleaning_status[task]['last_done_days_ago']
        new_days_ago = 7 - entry['day']
        if new_days_ago < prev_days_ago:
            user_cleaning_status[task]['last_done_days_ago'] = new_days_ago
        not_done.discard(task)

    for task in not_done:
        user_cleaning_status[task]['last_done_days_ago'] += 7 # 이번주에 안 한 청소는 'last_done_days_ago' 7일 증가

    return user_cleaning_status


def compute_user_behavior(prev_week_execution, prev_behavior=None, alpha=0.5):
    vector = np.zeros(168)
    for entry in prev_week_execution:
        index = entry['day'] * 24 + entry['hour']
        if entry['performed']:
            vector[index] += 1.0

    if vector.sum() > 0:
        vector /= vector.sum()
    vector = vector.astype(np.float32)

    if prev_behavior is None:
        return vector
    else:
        return alpha * prev_behavior + (1 - alpha) * vector

def preprocessing_schedules(week_start, raw_schedules):
    week_end = week_start + timedelta(days=7)
    fixed_schedules = []
    for item in raw_schedules:
        start_dt = item['start']['dateTime']
        end_dt = item['end']['dateTime']

        # +09:00 제거
        start_dt = datetime.fromisoformat(start_dt[:-6])
        end_dt = datetime.fromisoformat(end_dt[:-6])

        # 여러 날에 걸친 일정을 하루 단위로 분리
        current = start_dt
        while current.date() <= end_dt.date():
            day_start = current.replace(hour=0, minute=0, second=0, microsecond=0)
            day_end = day_start + timedelta(days=1)

            seg_start = max(start_dt, day_start)
            seg_end = min(end_dt, day_end)

            if not (week_start <= seg_start.date() < week_end):
                current = day_end
                continue

            weekday = seg_start.weekday() # 요일 0=월, 6=일
            start_hour = seg_start.hour + seg_start.minute / 60
            end_hour = seg_end.hour + seg_end.minute / 60

            fixed_schedules.append({
                'summary': item['summary'],
                'location': item['location'],
                'day': weekday,
                'start': start_hour,
                'end': end_hour
            })

            current = day_end

    return fixed_schedules

def prepare_user_state(user_execution_data, this_week, user_cleaning_status):
    prev_week_execution = user_execution_data.get(this_week-1, [])
    user_cleaning_status = compute_user_cleaning_status(user_cleaning_status, prev_week_execution)
    user_behavior = compute_user_behavior(prev_week_execution)
    if user_behavior is None:
        raise ValueError("user_behavior is None")
    return user_cleaning_status, user_behavior

### ilp_scheduler.py  
* generate_initial_schedule


In [8]:
def generate_initial_schedule(cleaning_tasks, user_cleaning_status, fixed_schedules,
                              total_cleaning_task_limit=10, total_cleaning_time_limit=5):
    # 1) ILP 문제 정의
    prob = pulp.LpProblem("CleaningScheduler", pulp.LpMaximize)

    # 2) 변수: 각 청소를 x요일 x시에 할지 말지 정함
    variables = {}
    for i, task in enumerate(cleaning_tasks):
        for d in range(7):  # 0 ~ 6(월요일~일요일)
            for h in range(24):  # 청소 시작시간
                variables[(i, d, h)] = pulp.LpVariable(f'task_{i}_day{d}_hour{h}', 0, 1, cat='Binary')

    # 3) 목적 함수 : 'interval', 'last_done_days_ago' 만족 -> 우선순위가 높은(미룬지 오래된) 청소 작업을 더 많이 포함
    objective = pulp.lpSum(
        (
            (
                (user_cleaning_status[task_name]["last_done_days_ago"] - task['interval'])
                + (7.0 if task['interval'] <= 3 else 0.0)
            ) / task['interval'] * variables[(i, d, h)]
        )
        for i, task_dict in enumerate(cleaning_tasks)
        for task_name, task in task_dict.items()
        for d in range(7) for h in range(24)
    )
    prob += objective

    # 4) 제약 조건
    # 제약 조건 안에서 최대한 많은 청소를 배정
    prob += pulp.lpSum([
        variables[(i, d, h)]
        for i in range(len(cleaning_tasks))
        for d in range(7) for h in range(24)])

    # 총 청소 개수 제한
    prob += pulp.lpSum([
        variables[(i, d, h)]
        for i in range(len(cleaning_tasks))
        for d in range(7) for h in range(24)]) <= total_cleaning_task_limit

    # 총 청소 시간 제한
    prob += pulp.lpSum([
        variables[(i, d, h)] * task['duration']
        for i, task_dict in enumerate(cleaning_tasks)
        for task_name, task in task_dict.items()
        for d in range(7) for h in range(24)]) <= total_cleaning_time_limit

    # 고정 일정 제외한 시간에만 청소 배정
    for i, task_dict in enumerate(cleaning_tasks):
        for task_name, task in task_dict.items():
            duration = task["duration"]
            for d in range(7):
                for h in range(24):
                    for sched in fixed_schedules:
                        if sched['day'] == d:
                            if sched['start'] < h + duration and sched['end'] > h:
                                prob += variables[(i, d, h)] == 0

    # 같은 시간대(x시:00분)에 실행되는 청소의 duration 합 <= 1시간
    for d in range(7):
        for h in range(24):
            prob += pulp.lpSum([
                variables[(i, d, h)] * task["duration"]
                for i, task_dict in enumerate(cleaning_tasks)
                for task in task_dict.values()
            ]) <= 1

    # 각 청소작업은 1주일에 (7 // interval)회 이하만 추천
    for i, task_dict in enumerate(cleaning_tasks):
        for task_name, task in task_dict.items():
            max_assign = max(1, 7 // task["interval"])
            prob += pulp.lpSum([
                variables[(i, d, h)]
                for d in range(7) for h in range(24)
            ]) <= max_assign

    # 최적화 수행
    prob.solve()

    cleaning_schedules = []
    for i, task_dict in enumerate(cleaning_tasks):
        for task_name, task in task_dict.items():
            for d in range(7):
                for h in range(24):
                    if pulp.value(variables[(i, d, h)]) == 1:
                        cleaning_schedules.append((task_name, d, h))

    cleaning_schedules.sort(key=lambda x: (x[1], x[2])) # 요일, 시간 정렬
    return cleaning_schedules

### envs.py
*   CleaningEnv

In [46]:
class CleaningEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        fixed_schedules,
        cleaning_tasks,
        user_cleaning_status,
        user_behavior,
        user_execution_data,
        user_pref,
        slot_penalties,
        initial_schedules=None,
        user_feedback=None,
    ):
        super().__init__()

        self.fixed_schedules = fixed_schedules
        self.cleaning_tasks = cleaning_tasks
        self.user_cleaning_status = user_cleaning_status
        self.behavior_vector = np.asarray(user_behavior, dtype=np.float32)
        self.user_execution_data = user_execution_data
        self.slot_penalties = slot_penalties
        self.initial_schedules = initial_schedules or []
        self.survey_responses = user_pref
        self.user_feedback = user_feedback

        self.task_id2name = []
        self.task_name2meta = {}
        for i, d in enumerate(self.cleaning_tasks):
            name = list(d.keys())[0]
            self.task_id2name.append(name)
            self.task_name2meta[name] = d[name]

        self.n_tasks = len(self.cleaning_tasks)
        self.current_step = 0
        self.max_steps = 32

        self.action_space = spaces.MultiDiscrete([self.n_tasks, 7, 24]) # action: (task_idx, day, hour)
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(168 * 3,), dtype=np.float32)

        self.state = None                   # (504,)
        self.weekly_todo = []               # 최종 결과: [{title, day, hour}, ...]
        self.used_slots = set()             # (day, hour) 중복 방지

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed); random.seed(seed)
        super().reset(seed=seed)

        self.current_step = 0
        self.weekly_todo = []
        self.used_slots = set()
        self.state = self._get_initial_state()

        if self.user_execution_data:
            self.state[168:168+168] = np.maximum(0.0, self.state[168:336] - self.slot_penalties)

        return self.state.copy().astype(np.float32), {}

    def _get_initial_state(self):
        fixed_vector = np.zeros(168, dtype=np.float32)
        urgency_vector = np.zeros(168, dtype=np.float32)

        # 1) 고정 일정 마스킹
        for sched in self.fixed_schedules:
            start = int(sched['day'] * 24 + sched['start'])
            end = int(sched['day'] * 24 + sched['end'])
            start = max(0, min(167, start))
            end = max(0, min(168, end))
            if end > start:
                fixed_vector[start:end] = 1.0

        # 2) 급함도(urgency) 초기화: 각 task의 (last_done_days_ago / interval)을 168 슬롯에 균등 분산
        for d in self.cleaning_tasks:
            name = list(d.keys())[0]
            meta = d[name]
            interval = float(meta.get('interval', 7) or 0)  # None 방지
            last_done = float(self.user_cleaning_status.get(name, {}).get('last_done_days_ago', 0.0))
            if interval <= 0:
                urgency = 1.0
            else:
                urgency = min(1.0, last_done / (interval + 1e-8))
            urgency_vector += (urgency / 168.0)
        np.clip(urgency_vector, 0.0, 1.0, out=urgency_vector)

        # 3) behavior_vector 체크
        if self.behavior_vector.shape[0] != 168:
            raise ValueError(f"behavior_vector length must be 168, got {self.behavior_vector.shape[0]}")
        behavior_vector = np.clip(self.behavior_vector, 0.0, 1.0).astype(np.float32)
        state = np.concatenate([fixed_vector, urgency_vector, behavior_vector], axis=0).astype(np.float32)
        return state

    def step(self, action): # action: (task_idx, day, hour)
        task_idx, day, hour = int(action[0]), int(action[1]), int(action[2])
        day = int(np.clip(day, 0, 6))
        hour = int(np.clip(hour, 0, 23))
        task_name = self.task_id2name[task_idx]
        meta = self.task_name2meta[task_name]
        add_task = True

        # 주당 최대 배정 횟수 체크
        interval = int(meta.get('interval', 7) or 7)
        max_per_week = max(1, 7 // interval)
        assigned_count = sum(1 for (n, _, _) in self.weekly_todo if n == task_name)

        if assigned_count >= max_per_week:
            add_task = False

        # 사용된 슬롯 기록
        is_duplicate = (day, hour) in self.used_slots
        if not is_duplicate:
            self.used_slots.add((day, hour))

        # 고정 일정 충돌 검사
        for sched in self.fixed_schedules:
            if sched["day"] == day and sched["start"] <= hour < sched["end"]:
                add_task = False
                break

        # 상태 업데이트
        idx = day * 24 + hour
        self.state[168 + idx] = max(0.0, self.state[168 + idx] - 0.5) # urgency 감소

        # 스케줄 기록
        if add_task:
            self.weekly_todo.append((task_name, day, hour))

        # 보상 계산
        reward = self._calculate_reward(task_name, day, hour, meta, is_duplicate)

        # 종료 판정
        self.current_step += 1
        terminated = self.current_step >= self.max_steps
        truncated = False
        info = {}
        if terminated or truncated:
             self.weekly_todo_final = self.weekly_todo.copy()

        next_state = self.state.copy().astype(np.float32)
        return next_state, float(reward), terminated, truncated, info

    def _calculate_reward(self, task_name, day, start, meta, is_duplicate):
        slot_penalty = 0.0
        burden_penalty = 0.0
        urgency_score = 0.0
        frequency_score = 0.0
        preference_score = 0.0
        initial_hint = 0.0
        onboarding_bonus = 0.0
        feedback_score = 0.0

        idx = int(day * 24 + start)
        sr = self.survey_responses # 온보딩

        # 1) 슬롯 중복 패널티
        cd = sr['cleaning_distribution'] # [consistency, intensity]
        if is_duplicate:
            slot_penalty -= (2 * cd[0])

        # 2) 야간/새벽 부담
        if start < 7 or start > 22:
            burden_penalty -= 0.5

        # 3) 청소 급함도(많이 밀린 청소) & 자주 해야하는 청소
        last_done_days_ago = float(self.user_cleaning_status.get(task_name, {}).get("last_done_days_ago", 0.0))
        interval = float(meta.get("interval", 7) or 0.0)
        urgency_score += (last_done_days_ago - interval)
        frequency_score += (last_done_days_ago - interval + (7.0 if interval <= 4 else 0))

        # 4) 사용자 시간 선호도
        preference_score += float(self.behavior_vector[idx]) * 1.5

        # 5) ILP 결과와 유사하면 보상
        if self.initial_schedules:
            if (task_name, day, start) in set(self.initial_schedules):
                initial_hint += 3.0

        # 6) 선호 요일/시간 활용
        pt = sr['preferred_time_range'] # [start, end]
        pd = sr['preferred_days'] # []

        if pt[0] <= start < pt[1]:
            onboarding_bonus += 0.3
        else:
            onboarding_bonus -= 0.1

        if day in pd:
            onboarding_bonus += 0.3
        else:
            onboarding_bonus -= 0.1

        # 7) 사용자 피드백
        feedback_score += (self.user_feedback['cleaning_amount_score'] + self.user_feedback['recommended_time_score']) / 5.0

        return float(slot_penalty + urgency_score + frequency_score + burden_penalty + preference_score + initial_hint + onboarding_bonus + feedback_score)


### rl_utils.py    

*   MultiCategorical
*   multi_categorical_dist_fn
*   build_slot_penalty_all_weeks
*   create_cleaning_env_factory

In [10]:
class MultiCategorical(Distribution):
    arg_constraints = {}
    has_rsample = False

    def __init__(self, logits_list):
        self.categoricals = [Categorical(logits=logits) for logits in logits_list]
        batch_shape = self.categoricals[0].batch_shape
        event_shape = torch.Size([len(self.categoricals)])
        super().__init__(batch_shape=batch_shape, event_shape=event_shape, validate_args=False)

    def sample(self):
        return torch.stack([cat.sample() for cat in self.categoricals], dim=-1)

    def log_prob(self, actions):
        actions = actions.long()
        return torch.stack(
            [cat.log_prob(actions[..., i]) for i, cat in enumerate(self.categoricals)],
            dim=-1
        ).sum(dim=-1)

    def entropy(self):
        return torch.stack([cat.entropy() for cat in self.categoricals], dim=-1).sum(dim=-1)

    @property
    def mode(self):
        return torch.stack([cat.probs.argmax(dim=-1) for cat in self.categoricals], dim=-1)

    @property
    def variance(self):
        vars_ = []
        for cat in self.categoricals:
            p = cat.probs
            mean = (torch.arange(p.shape[-1], device=p.device) * p).sum(dim=-1)
            var = ((torch.arange(p.shape[-1], device=p.device) - mean.unsqueeze(-1)) ** 2 * p).sum(dim=-1)
            vars_.append(var)
        return torch.stack(vars_, dim=-1)

    @property
    def stddev(self):
        return self.variance.sqrt()

    @property
    def probs(self): # 선택사항
        return [cat.probs for cat in self.categoricals]

def multi_categorical_dist_fn(nvec):
    nvec = list(map(int, nvec))
    def dist_fn(logits):
        chunks = torch.split(logits, nvec, dim=-1)
        return MultiCategorical(chunks) # 단일 분포 객체로 감싸서 반환
    return dist_fn

def build_slot_penalty_all_weeks(user_execution_data: dict, decay: float = 0.9) -> np.ndarray:
    slot_penalty = np.zeros(168, dtype=float)
    if not user_execution_data:
        return slot_penalty

    weeks_sorted = sorted(user_execution_data.keys())
    weight_by_week = {}
    last_week = weeks_sorted[-1]
    for w in weeks_sorted:
        weight_by_week[w] = decay ** (last_week - w)

    for w in weeks_sorted:
        weight = weight_by_week[w]
        for entry in user_execution_data[w]:
            if entry.get("performed"):
                idx = int(entry["day"] * 24 + entry["hour"])
                slot_penalty[idx] += 0.25 * weight

    np.clip(slot_penalty, 0.0, 1.0, out=slot_penalty)
    return slot_penalty

def create_cleaning_env_factory(
    fixed_schedules,
    cleaning_tasks,
    user_cleaning_status,
    user_behavior,
    user_execution_data,
    user_pref,
    cleaning_schedules,
    user_feedback,
):

    slot_penalties = build_slot_penalty_all_weeks(user_execution_data, decay=0.9)

    def make_env():
        return CleaningEnv(
            fixed_schedules=fixed_schedules,
            cleaning_tasks=cleaning_tasks,
            user_cleaning_status=user_cleaning_status,
            user_behavior=user_behavior,
            user_execution_data=user_execution_data,
            user_pref=user_pref,
            slot_penalties=slot_penalties,
            initial_schedules=cleaning_schedules,
            user_feedback=user_feedback,
        )
    return make_env


### trainer.py

*   create_ppo_policy
*   update_schedule
*   make_schedule



In [11]:
def create_ppo_policy(env, device):
    state_shape = env.observation_space.shape or env.observation_space.n
    nvec = np.array(env.action_space.nvec, dtype=int)
    action_shape = int(nvec.sum())

    net_actor = Net(state_shape=state_shape, hidden_sizes=[128, 128], device=device, activation=nn.ReLU)
    actor = Actor(net_actor, action_shape=action_shape, device=device).to(device)
    critic = Critic(Net(state_shape=state_shape, hidden_sizes=[128, 128], device=device, activation=nn.ReLU), device=device).to(device)

    optim = torch.optim.Adam(list(actor.parameters()) + list(critic.parameters()), lr=3e-4)
    dist_fn = multi_categorical_dist_fn(env.action_space.nvec)
    policy = PPOPolicy(
        actor=actor,
        critic=critic,
        optim=optim,
        dist_fn=dist_fn,
        action_space=env.action_space,
        max_grad_norm=0.5,
        vf_coef=0.5,
        ent_coef=0.01,
        reward_normalization=False,
        advantage_normalization=True,
        eps_clip=0.2,
        action_scaling=False,
    )
    return policy

def update_schedule(
    fixed_schedules,
    cleaning_schedules,
    cleaning_tasks,
    user_execution_data,
    this_week,
    prev_behavior,
    user_cleaning_status,
    user_feedback=None,
    log_dir=None,
    SEED=None,
):
    if SEED:
        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(SEED)
            torch.cuda.manual_seed_all(SEED)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

    # 사용자 상태 준비
    user_cleaning_status, user_behavior = prepare_user_state(user_execution_data, this_week, user_cleaning_status)

    # 환경 팩토리
    make_env = create_cleaning_env_factory(
        fixed_schedules=fixed_schedules,
        cleaning_tasks=cleaning_tasks,
        user_cleaning_status=user_cleaning_status,
        user_behavior=user_behavior,
        user_execution_data=user_execution_data,
        user_pref=user_info['survey_responses'],
        cleaning_schedules=cleaning_schedules,
        user_feedback=user_feedback,
    )

    env = make_env()

    train_env_num, test_env_num = 8, 4
    train_envs = DummyVectorEnv([make_env for _ in range(train_env_num)])
    test_envs = DummyVectorEnv([make_env for _ in range(test_env_num)])
    train_envs.seed(SEED)
    test_envs.seed(SEED)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    policy = create_ppo_policy(env, device)
    train_collector = Collector(policy, train_envs)
    test_collector = Collector(policy, test_envs)

    reward_history = deque(maxlen=5)
    def stop_fn(mean_rewards: float) -> bool:
        reward_history.append(mean_rewards)
        moving_avg = np.mean(reward_history)
        # print(f"[stop_fn] mean_rewards={mean_rewards:.3f} | moving_avg(5)={moving_avg:.3f}")
        return len(reward_history) == reward_history.maxlen and moving_avg >= 500.0

    trainer = OnpolicyTrainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=10,
        step_per_epoch=1024,
        step_per_collect=256,
        repeat_per_collect=4,
        episode_per_test=test_env_num,
        batch_size=64,
        stop_fn=stop_fn,
        logger=None,
        test_in_train=True,
        show_progress=False,
        verbose=False,
    )
    result = trainer.run()
    # print("학습 완료! 결과:", result)

    # 평가
    eval_envs = DummyVectorEnv([make_env])
    eval_envs.seed(SEED)
    eval_collector = Collector(policy, eval_envs)
    eval_collector.collect(n_episode=1, reset_before_collect=True)

    weekly_todo = eval_collector.env.get_env_attr("weekly_todo_final")[0]
    weekly_todo.sort(key=lambda x: (x[1], x[2]))
    return weekly_todo, user_behavior

# 1주일 분량 일정을 만드는 함수
def make_schedule(
    this_week: int,
    week_start: date,
    goocal_data: List[Dict],
    sleep_data: List[Dict],
    user_execution_data: Dict[int, List[Dict]],
    prev_behavior=None,
    user_cleaning_status=None,
    user_feedback=None,
):
    global cleaning_tasks

    goocal_data.extend(sleep_data)
    fixed_schedules = preprocessing_schedules(week_start, goocal_data)

    # 초기 청소 일정
    initial_schedules = generate_initial_schedule(cleaning_tasks, user_cleaning_status, fixed_schedules)

    cleaning_schedules, user_behavior = [], None
    if this_week > 1: # 1주치 이상의 사용자 청소 기록 데이터가 쌓이면 강화학습에 필요한 데이터를 만들고 학습을 수행한다
        cleaning_schedules, user_behavior = update_schedule(
            fixed_schedules=fixed_schedules,
            cleaning_schedules=initial_schedules,
            cleaning_tasks=cleaning_tasks,
            user_execution_data=user_execution_data,
            this_week=this_week,
            prev_behavior=prev_behavior,
            user_cleaning_status=user_cleaning_status,
            user_feedback=user_feedback,
            log_dir=None,
            SEED=42,
        )
    else:
        cleaning_schedules = initial_schedules.copy()

    prev_behavior = user_behavior

    return initial_schedules, cleaning_schedules, prev_behavior

### experiment_utils.py



*   generate_sleeping_data
*   build_user_execution_data
*   build_user_feedback
*   generate_raw_schedules



In [26]:
from datetime import date, datetime, time, timedelta
from typing import List, Tuple, Dict, Optional
from collections import defaultdict
import hashlib
import random
import numpy as np

def generate_sleeping_data(
    base_date: date,
    sleep_at: int,   # 예: 22
    wake_at: int,    # 예: 7
    seed: Optional[int] = None
) -> List[Dict]:

    sleeping_data = []
    n_days = 7

    for i in range(-1, n_days):
        day = base_date + timedelta(days=i)
        if sleep_at < wake_at:
            start_dt = datetime.combine(day, time(sleep_at, 0))
            end_dt = datetime.combine(day, time(wake_at, 0))
            sleeping_data.append({
                "summary": "수면",
                "location": "집",
                "start": {"dateTime": start_dt.isoformat(timespec="minutes") + "+09:00"},
                "end": {"dateTime": end_dt.isoformat(timespec="minutes") + "+09:00"}
            })
        else:
            start_dt = datetime.combine(day, time(sleep_at, 0))
            end_dt = datetime.combine(day + timedelta(days=1), time(wake_at, 0))
            sleeping_data.append({
                "summary": "수면",
                "location": "집",
                "start": {"dateTime": start_dt.isoformat(timespec="minutes") + "+09:00"},
                "end": {"dateTime": end_dt.isoformat(timespec="minutes") + "+09:00"}
            })
    return sleeping_data

def build_user_execution_data(
    last_week_todo: List[Tuple[str, int, int]],
    week: int = 1,
    min_adherence: float = 1.0,
    seed: Optional[int] = None
) -> Tuple[Dict[int, List[Dict]], float]:

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    total = len(last_week_todo)
    if total == 0:
        return {}, 1.0

    min_adherence = max(0.0, min(1.0, float(min_adherence)))
    min_to_execute = int(np.ceil(total * min_adherence))
    execute_count = random.randint(min_to_execute, total) if total > min_to_execute else total

    indices = list(range(total))
    random.shuffle(indices)
    executed_idx = set(indices[:execute_count])

    grouped_data = defaultdict(list)

    for i, entry in enumerate(last_week_todo):
        if not (isinstance(entry, tuple) and len(entry) == 3):
            raise ValueError(f"Expected tuple (task_name, day, hour), but got: {entry!r}")

        task_name, day, start_hour = entry
        grouped_data[week].append({
            "task": str(task_name),
            "performed": (i in executed_idx),
            "day": int(day),
            "hour": int(start_hour)
        })

    # adherence = sum(1 for x in grouped_data[week] if x["performed"]) / total
    return dict(grouped_data)

def build_user_feedback(week_start_date):
    user_feedback = {
        "week_start_date": week_start_date,
        "cleaning_amount_score": 4,
        "recommended_time_score": 5,
        "comment": ""
    }

    return user_feedback

def _seed_from_year_week(base_date: date, salt: str = "") -> int:
    iso_year, iso_week, _ = base_date.isocalendar()
    key = f"{iso_year}-W{iso_week}-{salt}".encode("utf-8")
    digest = hashlib.sha256(key).digest()
    return int.from_bytes(digest[:8], "big")  # 64비트 정수 시드

def _seed_from_year(base_date: date, salt: str = "") -> int:
    iso_year = base_date.isocalendar()[0]
    key = f"{iso_year}-{salt}".encode("utf-8")
    digest = hashlib.sha256(key).digest()
    return int.from_bytes(digest[:8], "big")

def _slot_to_hm(slot: int, slots_per_hour: int) -> (int, int):
    hour = slot // slots_per_hour
    minute = (slot % slots_per_hour) * (60 // slots_per_hour)
    return hour, minute

def _hm_to_slot(hour: int, minute: int, slots_per_hour: int) -> int:
     return hour * slots_per_hour + minute // (60 // slots_per_hour)

def generate_raw_schedules(
    base_date: date,
    seed: Optional[int] = None,
    tz_offset: str = "+09:00",
    salt: str = "korean_univ_18cr_v1",
) -> List[Dict]:

    seed_val_week = _seed_from_year_week(base_date, salt) if seed is None else seed
    rng = random.Random(seed_val_week)

    seed_val_year_for_class = _seed_from_year(base_date, salt + "_class")
    rng_class = random.Random(seed_val_year_for_class)

    monday = base_date - timedelta(days=base_date.weekday())  # 0=월요일

    slot_minutes = 30
    slots_per_hour = 60 // slot_minutes           # 2
    slots_per_day = 24 * slots_per_hour           # 48

    occupancy = {d: [False] * slots_per_day for d in range(7)}
    events: List[Dict] = []

    num_courses = 6
    meetings_per_course = 2
    course_duration_slots = int(1.5 * slots_per_hour)  # 3슬롯 = 1.5시간

    class_start_earliest = _hm_to_slot(9, 0, slots_per_hour)
    class_end_latest = _hm_to_slot(18, 0, slots_per_hour)  # 끝 시각 한계
    lunch_start = _hm_to_slot(12, 0, slots_per_hour)
    lunch_end = _hm_to_slot(13, 0, slots_per_hour)

    def can_place(d: int, start_slot: int, dur_slots: int) -> bool:
        end_slot = start_slot + dur_slots
        if end_slot > slots_per_day:
            return False
        if not (end_slot <= lunch_start or start_slot >= lunch_end):
            return False
        # 점유 확인
        return not any(occupancy[d][start_slot:end_slot])

    def place_event(
        title: str,
        d: int,
        start_slot: int,
        dur_slots: int,
        location: str,
        recurrence: Optional[List[str]] = None,
    ):
        """이벤트를 점유표에 반영하고 events에 추가"""
        for s in range(start_slot, start_slot + dur_slots):
            occupancy[d][s] = True
        sh, sm = _slot_to_hm(start_slot, slots_per_hour)
        start_dt = datetime.combine(monday + timedelta(days=d), time(hour=sh, minute=sm))
        end_dt = start_dt + timedelta(minutes=dur_slots * slot_minutes)
        event = {
            "summary": title,
            "location": location,
            "start": {"dateTime": start_dt.isoformat(timespec="minutes") + tz_offset},
            "end": {"dateTime": end_dt.isoformat(timespec="minutes") + tz_offset},
        }
        if recurrence:
            event["recurrence"] = recurrence
        events.append(event)

    for i in range(1, num_courses + 1):
        title = f"수업{i}"
        two_days = rng_class.sample(range(0, 5), k=meetings_per_course)

        for d in two_days:
            placed = False
            latest_start = class_end_latest - course_duration_slots
            start_candidates = list(range(class_start_earliest, latest_start + 1))
            rng_class.shuffle(start_candidates)

            for start_slot in start_candidates:
                if can_place(d, start_slot, course_duration_slots):
                    place_event(
                        title, d, start_slot, course_duration_slots,
                        location="캠퍼스",
                        recurrence=["RRULE:FREQ=WEEKLY"]
                    )
                    placed = True
                    break
            if not placed:
                for start_slot in start_candidates:
                    end_slot = start_slot + course_duration_slots
                    if end_slot <= slots_per_day and not any(occupancy[d][start_slot:end_slot]):
                        place_event(
                            title, d, start_slot, course_duration_slots,
                            location="캠퍼스",
                            recurrence=["RRULE:FREQ=WEEKLY"]
                        )
                        placed = True
                        break
            if not placed:
                for start_slot in range(0, slots_per_day - course_duration_slots + 1):
                    end_slot = start_slot + course_duration_slots
                    if not any(occupancy[d][start_slot:end_slot]):
                        place_event(
                            title, d, start_slot, course_duration_slots,
                            location="캠퍼스",
                            recurrence=["RRULE:FREQ=WEEKLY"]
                        )
                        placed = True
                        break

    meal_titles = ["밥약1", "밥약2"]
    meal_duration_slots = int(2 * slots_per_hour)  # 2시간 = 4슬롯

    lunch_start_min = _hm_to_slot(11, 0, slots_per_hour)
    lunch_start_max = _hm_to_slot(13, 0, slots_per_hour)

    dinner_start_min = _hm_to_slot(18, 0, slots_per_hour)
    dinner_start_max = _hm_to_slot(20, 0, slots_per_hour)

    for title in meal_titles:
        meal_type = rng.choice(["lunch", "dinner"])
        day_candidates = list(range(0, 7))
        rng.shuffle(day_candidates)
        placed = False

        for d in day_candidates:
            if meal_type == "lunch":
                start_candidates = list(range(lunch_start_min, lunch_start_max + 1))
                loc = "식당"
            else:
                start_candidates = list(range(dinner_start_min, dinner_start_max + 1))
                loc = "식당"
            rng.shuffle(start_candidates)

            for start_slot in start_candidates:
                end_slot = start_slot + meal_duration_slots
                if end_slot <= slots_per_day and not any(occupancy[d][start_slot:end_slot]):
                    place_event(title, d, start_slot, meal_duration_slots, location=loc)
                    placed = True
                    break
            if placed:
                break
        if not placed:
            for d in range(7):
                for start_slot in range(dinner_start_min, slots_per_day - meal_duration_slots + 1):
                    end_slot = start_slot + meal_duration_slots
                    if not any(occupancy[d][start_slot:end_slot]):
                        place_event(title, d, start_slot, meal_duration_slots, location="식당")
                        placed = True
                        break
                if placed:
                    break

    workout_titles = ["운동1", "운동2"]
    workout_duration_slots = int(1 * slots_per_hour)  # 1시간 = 2슬롯
    workout_start_min = _hm_to_slot(20, 0, slots_per_hour)
    workout_start_max = _hm_to_slot(23, 0, slots_per_hour) - workout_duration_slots  # 23:00 이전 시작

    for title in workout_titles:
        day_candidates = list(range(0, 7))
        rng.shuffle(day_candidates)
        placed = False

        for d in day_candidates:
            start_candidates = list(range(workout_start_min, workout_start_max + 1))
            rng.shuffle(start_candidates)
            for start_slot in start_candidates:
                end_slot = start_slot + workout_duration_slots
                if end_slot <= slots_per_day and not any(occupancy[d][start_slot:end_slot]):
                    place_event(title, d, start_slot, workout_duration_slots, location="체육관")
                    placed = True
                    break
            if placed:
                break
        if not placed:
            relax_min = _hm_to_slot(19, 0, slots_per_hour)
            relax_max = _hm_to_slot(23, 30, slots_per_hour) - workout_duration_slots
            for d in range(7):
                for start_slot in range(relax_min, max(relax_min, relax_max) + 1):
                    end_slot = start_slot + workout_duration_slots
                    if end_slot <= slots_per_day and not any(occupancy[d][start_slot:end_slot]):
                        place_event(title, d, start_slot, workout_duration_slots, location="체육관")
                        placed = True
                        break
                if placed:
                    break

    events.sort(key=lambda ev: ev["start"]["dateTime"])
    return events



### experiment.py

* convert_to_weeklyTodos
* run_one_week

In [39]:
from datetime import timedelta
import uuid

def convert_to_weeklyTodos(weekly_todo, week_start):
    by_day = {d: [] for d in range(7)}
    for task_name, day, hour in weekly_todo:
        by_day[day].append((task_name, hour))

    for d in range(7):
        by_day[d].sort(key=lambda x: (x[1], x[0]))

    def new_id():
        return uuid.uuid4().hex[:6]

    def to_hhmm(hour: int):
        return f"{hour:02d}:00"

    weeklyTodos = []
    for d in range(7):
        date_str = (week_start + timedelta(days=d)).strftime("%Y-%m-%d")
        todos = [
            {
                "id": new_id(),
                "title": task_name,
                "startHour": to_hhmm(hour),
                "isDone": False
            }
            for task_name, hour in by_day[d]
        ]
        weeklyTodos.append({"date": date_str, "todos": todos})

    return weeklyTodos


def run_one_week(this_week: int, week_start: date, last_week_todo=None, prev_behavior=None, user_cleaning_status=None):

    raw_schedules = generate_raw_schedules(week_start) # 구글 캘린더

    sleep_at, wake_at = user_info['survey_responses']['sleep_at'], user_info['survey_responses']['wake_at'] # ok
    sleeping = generate_sleeping_data(week_start, sleep_at=sleep_at, wake_at=wake_at)

    user_feedback = None
    if this_week > 1:
        user_feedback = build_user_feedback(week_start)
        prev_user_exec = build_user_execution_data(
            last_week_todo=last_week_todo,
            week=this_week-1,
            seed=body.seed
        )
        user_execution_data.update(prev_user_exec)


    initial_todo, weekly_todo, prev_behavior = make_schedule(
        this_week=this_week,
        week_start=week_start,
        goocal_data=raw_schedules,
        sleep_data=sleeping,
        user_execution_data=user_execution_data,
        prev_behavior=prev_behavior,
        user_cleaning_status=user_cleaning_status,
        user_feedback=user_feedback,
    )

    behavior_vector = None
    if this_week > 1:
        behavior_vector = compute_user_behavior(prev_user_exec[this_week-1])

    return behavior_vector, weekly_todo


### main.py

In [52]:
from typing import Any, Dict, List, Optional
from datetime import date, timedelta
from fastapi import FastAPI
from pydantic import BaseModel, Field



app = FastAPI(title="AI-SS API", version="1.0.0")

class OneWeekInput(BaseModel):
    week_start: date
    seed: int = 42

# -------- Endpoint --------
base_date = None
behavior_history = {}
weekly_todo_history = {}

@app.post("/api/v1/schedule/run-one-week")
def generate_one_week_schedule(body: OneWeekInput):
    global base_date

    if base_date is None:
        base_date = body.week_start

    delta_days = (body.week_start - base_date).days
    this_week = (delta_days // 7) + 1

    prev_behavior = behavior_history.get(body.week_start - timedelta(days=7), None)
    last_week_todo = weekly_todo_history.get(body.week_start - timedelta(days=7), None)

    behavior_vector, weekly_todo = run_one_week(
        this_week, body.week_start, last_week_todo, prev_behavior, user_cleaning_status
    )

    behavior_history[body.week_start] = behavior_vector # 파일에 쓰기
    weekly_todo_history[body.week_start] = weekly_todo # 파일에 쓰기

    weeklyTodos = convert_to_weeklyTodos(weekly_todo, body.week_start) # 리턴 전용 형식으로 변환

    return {"weeklyTodos": weeklyTodos}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [53]:
body = OneWeekInput(
    week_start="2025-10-13",
)
resp = generate_one_week_schedule(body)
from pprint import pprint
pprint(resp["weeklyTodos"])


/usr/local/lib/python3.12/dist-packages/pulp/pulp.py:1865: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


[{'date': '2025-10-13',
  'todos': [{'id': '242a55',
             'isDone': False,
             'startHour': '22:00',
             'title': '전자기기 소독'}]},
 {'date': '2025-10-14', 'todos': []},
 {'date': '2025-10-15',
  'todos': [{'id': '5f162d',
             'isDone': False,
             'startHour': '22:00',
             'title': '화장실 - 변기 청소'}]},
 {'date': '2025-10-16',
  'todos': [{'id': '1a5858',
             'isDone': False,
             'startHour': '13:00',
             'title': '화장실 - 거울 닦기'},
            {'id': '882ddb',
             'isDone': False,
             'startHour': '15:00',
             'title': '쓰레기 배출'},
            {'id': 'b25163',
             'isDone': False,
             'startHour': '15:00',
             'title': '화장실 - 세면대 청소'}]},
 {'date': '2025-10-17', 'todos': []},
 {'date': '2025-10-18',
  'todos': [{'id': 'f78461',
             'isDone': False,
             'startHour': '09:00',
             'title': '화장실 - 바닥 청소'},
            {'id': 'f7b569',
         

In [54]:
body = OneWeekInput(
    week_start="2025-10-20",
)
resp = generate_one_week_schedule(body)
from pprint import pprint
pprint(resp["weeklyTodos"])


[{'date': '2025-10-20',
  'todos': [{'id': '206a15',
             'isDone': False,
             'startHour': '12:00',
             'title': '주방 - 냉장고 유통기한 지난 음식 1개 이상 처분'},
            {'id': '90ab5a',
             'isDone': False,
             'startHour': '12:00',
             'title': '주방 - 수건 교체'},
            {'id': 'aa3237',
             'isDone': False,
             'startHour': '19:00',
             'title': '주방 - 배기후드 기름떼 제거'}]},
 {'date': '2025-10-21', 'todos': []},
 {'date': '2025-10-22',
  'todos': [{'id': '83bfde',
             'isDone': False,
             'startHour': '08:00',
             'title': '거실 - 바닥 물걸레질'},
            {'id': '724ca0',
             'isDone': False,
             'startHour': '17:00',
             'title': '거실 - 테이블 위 정리'},
            {'id': 'c094d7',
             'isDone': False,
             'startHour': '21:00',
             'title': '화장실 - 변기 청소'},
            {'id': '1f578f',
             'isDone': False,
             'startHour': '22:00',
  

In [55]:
body = OneWeekInput(
    week_start="2025-10-27",
)
resp = generate_one_week_schedule(body)
from pprint import pprint
pprint(resp["weeklyTodos"])


[{'date': '2025-10-27',
  'todos': [{'id': '6e08ed',
             'isDone': False,
             'startHour': '12:00',
             'title': '주방 - 냉장고 유통기한 지난 음식 1개 이상 처분'},
            {'id': 'd6f596',
             'isDone': False,
             'startHour': '12:00',
             'title': '주방 - 수건 교체'},
            {'id': '565592',
             'isDone': False,
             'startHour': '19:00',
             'title': '주방 - 배기후드 기름떼 제거'},
            {'id': '3a8194',
             'isDone': False,
             'startHour': '20:00',
             'title': '거실 - 테이블 위 정리'},
            {'id': '29395e',
             'isDone': False,
             'startHour': '22:00',
             'title': '화장실 - 바닥 청소'}]},
 {'date': '2025-10-28', 'todos': []},
 {'date': '2025-10-29',
  'todos': [{'id': 'cc0b60',
             'isDone': False,
             'startHour': '08:00',
             'title': '거실 - 바닥 물걸레질'},
            {'id': '7b37ae',
             'isDone': False,
             'startHour': '17:00',
  

In [60]:
body = OneWeekInput(
    week_start="2025-11-03",
)
resp = generate_one_week_schedule(body)
from pprint import pprint
pprint(resp["weeklyTodos"])


/usr/local/lib/python3.12/dist-packages/pulp/pulp.py:1865: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


[{'date': '2025-11-03',
  'todos': [{'id': '3b7425',
             'isDone': False,
             'startHour': '12:00',
             'title': '주방 - 냉장고 유통기한 지난 음식 1개 이상 처분'},
            {'id': '2841a5',
             'isDone': False,
             'startHour': '12:00',
             'title': '주방 - 수건 교체'},
            {'id': 'ebac47',
             'isDone': False,
             'startHour': '19:00',
             'title': '주방 - 배기후드 기름떼 제거'},
            {'id': '45ef79',
             'isDone': False,
             'startHour': '20:00',
             'title': '거실 - 테이블 위 정리'},
            {'id': 'd95465',
             'isDone': False,
             'startHour': '22:00',
             'title': '화장실 - 바닥 청소'}]},
 {'date': '2025-11-04', 'todos': []},
 {'date': '2025-11-05',
  'todos': [{'id': 'c7737d',
             'isDone': False,
             'startHour': '08:00',
             'title': '거실 - 바닥 물걸레질'},
            {'id': 'c0d5d8',
             'isDone': False,
             'startHour': '17:00',
  

In [59]:
pprint(user_execution_data)

{1: [{'day': 0, 'hour': 22, 'performed': True, 'task': '전자기기 소독'},
     {'day': 2, 'hour': 22, 'performed': True, 'task': '화장실 - 변기 청소'},
     {'day': 3, 'hour': 13, 'performed': True, 'task': '화장실 - 거울 닦기'},
     {'day': 3, 'hour': 15, 'performed': True, 'task': '화장실 - 세면대 청소'},
     {'day': 3, 'hour': 15, 'performed': True, 'task': '쓰레기 배출'},
     {'day': 5, 'hour': 9, 'performed': True, 'task': '화장실 - 바닥 청소'},
     {'day': 5,
      'hour': 15,
      'performed': True,
      'task': '주방 - 냉장고 유통기한 지난 음식 1개 이상 처분'},
     {'day': 5, 'hour': 16, 'performed': True, 'task': '화장실 - 하수구 청소'},
     {'day': 5, 'hour': 21, 'performed': True, 'task': '주방 - 배기후드 기름떼 제거'},
     {'day': 6, 'hour': 20, 'performed': True, 'task': '세탁실 - 세탁기 세제통 씻기'}],
 2: [{'day': 0,
      'hour': 12,
      'performed': True,
      'task': '주방 - 냉장고 유통기한 지난 음식 1개 이상 처분'},
     {'day': 0, 'hour': 12, 'performed': True, 'task': '주방 - 수건 교체'},
     {'day': 0, 'hour': 19, 'performed': True, 'task': '주방 - 배기후드 기름떼 제거'},


## 내부 데이터 확인

In [61]:
print('\n=== 마지막 수행일로부터 경과된 날짜 ===')
lst = [item for item in user_cleaning_status.items()]
print('\n'.join(map(str, lst)))


=== 마지막 수행일로부터 경과된 날짜 ===
('주방 - 수건 교체', {'last_done_days_ago': 7})
('주방 - 행주 삶기', {'last_done_days_ago': 17})
('주방 - 배기후드 기름떼 제거', {'last_done_days_ago': 7})
('주방 - 냉장고 유통기한 지난 음식 1개 이상 처분', {'last_done_days_ago': 7})
('세탁실 - 세탁기 세제통 씻기', {'last_done_days_ago': 40})
('화장실 - 바닥 청소', {'last_done_days_ago': 7})
('화장실 - 변기 청소', {'last_done_days_ago': 5})
('화장실 - 세면대 청소', {'last_done_days_ago': 15})
('화장실 - 하수구 청소', {'last_done_days_ago': 17})
('화장실 - 거울 닦기', {'last_done_days_ago': 17})
('화장실 - 수납장 내부 정리 및 청소', {'last_done_days_ago': 5})
('거실 - 가구 표면 먼지 닦기 (물걸레)', {'last_done_days_ago': 1})
('거실 - 가구 표면 먼지 털기 (먼지털이)', {'last_done_days_ago': 17})
('거실 - 바닥 물걸레질', {'last_done_days_ago': 5})
('거실 - 청소기로 바닥 먼지 제거', {'last_done_days_ago': 4})
('거실 - 테이블 위 정리', {'last_done_days_ago': 5})
('거실 - 안 쓰는 물건 1개 이상 처분', {'last_done_days_ago': 20})
('창틀 먼지 닦기', {'last_done_days_ago': 3})
('전자기기 소독', {'last_done_days_ago': 15})
('쓰레기 배출', {'last_done_days_ago': 13})
('음식물 쓰레기 배출', {'last_done_days_ago':